In [ ]:
%pip install strux
# !pip install strux

In [1]:
from typing import List, Literal
import pandas as pd
from pydantic import BaseModel

from strux import PostgresDataSource, RegressionConfig, Sequential, exact_match

# Strux
Strux is a Python framework for structured outputs model versioning. It helps you:

- Track changes in your model outputs over time
- Detect regressions in model behavior
- Compare model outputs against baselines
- Validate outputs against schema definitions

This notebook demonstrates the basic usage of Strux using a simple sentiment analysis example.

In this example, we will:

1. Define input/output schemas
2. Define an inference function
3. Connect to a database
4. Run inference on a dataset
5. Compare the results to a baseline


We can define connection parameters in a dictionary and pass it to the `PostgresDataSource` constructor. The `PostgresDataSource` will be used to connect to the database and fetch the data.

It is recommended to use environment variables to store connection parameters.

In [2]:
import os 

# Connection Parameters
connection_params = {
    "user": os.getenv("POSTGRES_USER"),
    "password": os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("POSTGRES_HOST"),
    "port": os.getenv("POSTGRES_PORT"),
    "database": os.getenv("POSTGRES_DATABASE")
}

# Define input/output schemas
Strux supports Pydantic models as input/output schemas. 

The `InputSchema` is a Pydantic model that we will coerce the read output from the database into. In this scenario, each row from the database has a `chat_history` column that contains a list of `ChatMessage` objects stored as JSONB.

The `OutputSchema` is a Pydantic model that we will coerce the output of our inference function into. In this scenario, we will return a `sentiment` field that is a string.

In [3]:
# Define input/output schemas
class ChatMessage(BaseModel):
    role: Literal["user", "assistant"]
    content: str

class InputSchema(BaseModel):
    chat_history: List[ChatMessage]

class OutputSchema(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]

We can now define a `PostgresDataSource` that will fetch the data from the database and coerce it into the `InputSchema`.

In [5]:
source = PostgresDataSource(
    query="SELECT * FROM matador_s_idx LIMIT 1",
    connection_params=connection_params,
    schema=InputSchema,
    json_columns={"chat_history": List[ChatMessage]}
)

# Define inference function
Strux builds pipelines from steps. A "Step" is the smallest unit of work in Strux. It is a function that takes an input and returns an output where the input and output are both Pydantic models. 

For simplicity, we will define a single step that takes the input schema and returns the output schema. For generative AI models that produce structured outputs like OpenAI's API, this is the most common case.

In [4]:
def inference(input: InputSchema) -> OutputSchema:
    return OutputSchema(sentiment="positive")

# Define a pipeline
In Strux, a pipeline is a sequence of steps. We can define a pipeline by calling the `from_steps` method on the `Sequential` class. This method takes a list of steps and returns a pipeline.

In this scenario, we will define a pipeline that takes the input schema, runs inference on it, and returns the output schema.

We will also define a `RegressionConfig` that will be used to configure the pipeline. The `RegressionConfig` is a configuration object that we will use to configure the inference function. We will use the `exact_match` strategy to compare the output of our inference function to the expected output.

In [6]:
config = RegressionConfig(
    target_schema=OutputSchema,
    strict_fields=["sentiment"],
)

config.configure_field("sentiment", strategy=exact_match())

pipeline = Sequential.from_steps(
    data_source=source,
    steps=[("sentiment", inference, OutputSchema)],
    config=config
)

# Initial Run

In [7]:
def setup_pipeline(connection_params: dict) -> Sequential:
    source = PostgresDataSource(
        query="SELECT * FROM matador_s_idx LIMIT 1",
        connection_params=connection_params,
        schema=InputSchema,
        json_columns={"chat_history": List[ChatMessage]}
    )

    config = RegressionConfig(
        target_schema=OutputSchema,
        strict_fields=["sentiment"],
    )

    config.configure_field("sentiment", strategy=exact_match())

    return Sequential.from_steps(
        data_source=source,
        steps=[("sentiment", inference, OutputSchema)],
        config=config
    )


In [ ]:
pipeline = setup_pipeline(connection_params)

# First run - create baseline
results = pipeline.run(
    baseline_path="baselines/sentiment_baseline.json"
)

# Check if this is first run (no baseline)
is_first_run = all(
    "is_first_run" in step.metadata and step.metadata["is_first_run"]
    for step in results.step_validations
)

if is_first_run:
    print("\nFirst run completed. Would you like to save as baseline? (y/n)")
    if input().lower() == 'y':
        results.save_as_baseline("baselines/sentiment_baseline.json")
        print("\nNext steps:")
        print("1. Make changes to your model")
        print("2. Run regression test against baseline:")
        print("   pipeline.run(baseline_path='baselines/sentiment_baseline.json')")
else:
    if results.passed:
        print("\nRegression test passed. No changes needed.")
    else:
        print("\nRegression test failed. Please make changes to your model.")
        for step in results.get_failed_steps():
            print(f"\n{step.format_summary()}")

# Second run - compare to baseline
Suppose we make some changes to our model and want to run the pipeline again. We can do this by calling the `run` method on the pipeline again and passing in the baseline path. We can change the inference function directly in the pipeline definition or we can define a new inference function and pass it to the `run` method.

In this scenario, we will change the inference function directly in the pipeline definition.

In [ ]:
def inference(input: InputSchema) -> OutputSchema:
    return OutputSchema(sentiment="negative")

pipeline = pipeline.from_steps(
    data_source=source,
    steps=[("sentiment", inference, OutputSchema)],
    config=config
)

results = pipeline.run(
    baseline_path="baselines/sentiment_baseline.json"
)

if results.passed:
    print("\nRegression test passed. No changes needed.")
else:
    print("\nRegression test failed. Please make changes to your model.")
    for step in results.get_failed_steps():
        print(f"\n{step.format_summary()}")